In [17]:
# Import dependencies
import pandas as pd

### **Import and Display CSV**

In [18]:
# Filepath
csv_filepath =  "../data/raw/listings.csv"

# Read CSV
listings_csv = pd.read_csv(csv_filepath)

### **Removing Unnecessary/ Noisy Data Columns**

In [19]:
# list of Columns to Definitely Remove
definite_removal = ['listing_url', 'scrape_id', 'last_scraped', 'source','picture_url', 'host_id',
                    'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
                    'host_response_time', 'host_thumbnail_url', 'host_picture_url', 
                    'host_neighbourhood', 'host_total_listings_count', 'host_verifications', 
                    'host_has_profile_pic', 'host_identity_verified','minimum_minimum_nights', 
                    'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 
                    'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'calendar_updated',
                    'calendar_last_scraped', 'last_review', 'review_scores_accuracy', 
                    'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 
                    'license', 'instant_bookable','reviews_per_month']

# Potential Removal    
potential_removal = ['name', 'description', 'neighborhood_overview', 'host_response_rate', 
                    'host_acceptance_rate', 'neighbourhood', 'neighbourhood_group_cleansed', 'bathrooms',
                    'has_availability', 'availability_30', 'availability_60', 'availability_90',
                    'availability_365', 'availability_eoy', 'number_of_reviews', 'review_scores_rating',
                    'number_of_reviews_ltm', 'number_of_reviews_l30d', 'review_scores_value', 'review_scores_location',
                    'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 
                    'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms']

removal = definite_removal + potential_removal

listings_reduced = listings_csv.drop(removal, axis=1)

### **Drop Null Values/ Inactive Listings with null values**

In [20]:
# Copy dataframe
listings_cleaned = listings_reduced.copy()

# Fill 'na' for specific rows where number_of_reviews == 0 and first_review is null thus selecting all new listing (new listing = no reviews)
filtered_rows = (listings_cleaned['number_of_reviews_ly'] == 0) & (listings_cleaned['first_review'].isnull())

# Replace Empty values in review columns with 'na' only for unreviewed listings not the inactive listings
listings_cleaned.loc[filtered_rows, 'first_review'] = 'na'

# Now drop rows that still have nulls (i.e. truly missing data)
listings_cleaned = listings_cleaned.dropna(how='any')

# Drop first_review as we only needed it for this part
listings_cleaned= listings_cleaned.drop(columns='first_review')

### **Fix Data Types**

In [21]:
# fix datatypes
listings_cleaned['host_is_superhost'] = listings_cleaned['host_is_superhost'].map({'t':1, 'f':0}).astype('int64')
listings_cleaned['host_listings_count'] = listings_cleaned['host_listings_count'].astype('int64')
listings_cleaned['bedrooms'] = listings_cleaned['bedrooms'].astype('int64')
listings_cleaned['beds'] = listings_cleaned['beds'].astype('int64')
listings_cleaned['price'] = listings_cleaned['price'].str.replace('$', '').str.replace(',', '').astype('float')


### **Standardize and Bin bathroom_text and Rename  to bathrooms** 

In [22]:
# Bathroom value bins
no_bath = ['0 baths', '0 shared baths']
one_bath = ['1 bath', 'Half-bath']
one_shared_bath = ['1 shared bath', 'Shared half-bath']
one_private_bath = ['1 private bath', 'Private half-bath']
two_bath = ['2 baths', '2.5 baths', '1.5 baths', ]
two_shared_bath = ['1.5 shared baths', '2 shared baths', '2.5 shared baths',]
three_plus_bath = ['3 baths', '4 baths', '5 baths', '3.5 baths', '4.5 baths', '5.5 baths', '6.5 baths', '6 baths', '20 baths']
three_plus_shared_bath = ['3 shared baths', '3.5 shared baths', '4 shared baths', '4.5 shared baths']


# Function to group bathroom values
def simplify_bathroom_text(text):
    if text in no_bath:
        return 'no bath'
    elif text in one_shared_bath:
        return '1 shared bath'
    elif text in one_private_bath:
        return '1 private bath'
    elif text in one_bath:
        return '1 bath'
    elif text in two_shared_bath:
        return '2 shared bath'
    elif text in two_bath:
        return '2 bath'
    elif text in three_plus_bath:
        return '3 or more baths'
    elif text in three_plus_shared_bath:
        return '3 or more shared baths'
    else:
        return text

# Apply Function and update
listings_cleaned['bathrooms_text'] = listings_cleaned['bathrooms_text'].apply(simplify_bathroom_text)
listings_cleaned.rename(columns={'bathrooms_text': 'bathrooms'}, inplace=True)

### **Check and Save Cleaned Data**

In [23]:
# Check Output Data
listings_cleaned

,id,host_is_superhost,host_listings_count,neighbourhood_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,number_of_reviews_ly,estimated_occupancy_l365d,estimated_revenue_l365d
0,696407278180533419,0,7,Annex,43.670920,-79.395190,Entire condo,Entire home/apt,3,1 bath,1,1,"[""Hair dryer"", ""Central heating"", ""Paid parkin...",450.0,28,365,2,56,25200.0
1,696457318817239920,1,8,Kensington-Chinatown,43.655704,-79.399910,Private room in rental unit,Private room,2,1 private bath,1,1,"[""Hair dryer"", ""Central heating"", ""Shampoo"", ""...",78.0,28,365,2,112,8736.0
2,696460661218975906,1,8,Kensington-Chinatown,43.655492,-79.401834,Private room in rental unit,Private room,2,1 private bath,1,1,"[""Hair dryer"", ""Central heating"", ""Shampoo"", ""...",78.0,28,365,0,0,0.0
4,696602542310304703,1,1,Kensington-Chinatown,43.648778,-79.401183,Entire rental unit,Entire home/apt,3,1 bath,1,1,"[""Hair dryer"", ""Room-darkening shades"", ""Keypa...",132.0,28,120,4,224,29568.0
5,696638535490228478,0,1,Waterfront Communities-The Island,43.640220,-79.392420,Private room in condo,Private room,2,2 bath,2,2,"[""Pool table"", ""Dedicated workspace"", ""Exercis...",160.0,28,365,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21655,1366954101885217722,0,7,Moss Park,43.662840,-79.370420,Entire home,Entire home/apt,5,1 bath,2,2,"[""Heating"", ""Shampoo"", ""Iron"", ""Body soap"", ""B...",75.0,28,90,0,0,0.0
21656,1366970333908472070,0,1,Waterfront Communities-The Island,43.640498,-79.397431,Entire rental unit,Entire home/apt,5,1 bath,2,3,"[""Hair dryer"", ""Room-darkening shades"", ""Eleva...",197.0,2,365,0,0,0.0
21657,1366999672978459092,0,5,Downsview-Roding-CFB,43.736072,-79.483793,Entire rental unit,Entire home/apt,4,2 bath,2,2,"[""Pool table"", ""Fire extinguisher"", ""Outdoor d...",184.0,28,365,0,0,0.0
21658,1367290343089381102,0,2,Rouge,43.827085,-79.228557,Entire home,Entire home/apt,2,1 bath,1,1,"[""Keypad"", ""Heating"", ""Iron"", ""Body soap"", ""Be...",128.0,4,60,0,0,0.0


In [24]:
# Save Cleaned Data as CSV
listings_cleaned.to_csv('../data/processed/cleaned_listings.csv', index=False)
